<a href="https://colab.research.google.com/github/TK-Problem/Python-mokymai/blob/master/Scripts/cvonline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Importuoti paketus

# playwright biblioteka naudojama importuoti html kodą
!pip install playwright==1.25.00
!playwright install-deps
!playwright install webkit
!pip install nest_asyncio==1.5.6

# playwright veikia TIK asyncio režimu
import nest_asyncio
nest_asyncio.apply()
import asyncio

# importuoti playwright versiją
from playwright.async_api import async_playwright

# bsė naudojama iš HTML ištraukti reikiamą informaciją
from bs4 import BeautifulSoup

# kartais reikia palaikyti kurį laiką programą veikiančią
import time

# paketai dirbti su skaičiais ir duomenimis
import pandas as pd
import numpy as np

# clear output komanda naudojama išvalyti informacijai
from IPython.display import clear_output
clear_output()

# Duomenų atsisiuntimas

Funcijos veikimo žingsniai:

* sukuria `playwright` webdriver'į (webkit),
* sukuria netikrą `user_agent`, kad svetainė tave laikytų tikru varotoju,
* sugeneruoji `cvonline.lt` puslapio URL kartu su raktažodžių (keyword),
* paspaudžia ant pop-up ir cookie mygtukų,
* palaukia prevenciškai 2 sekundes,
* atsisiunčia HTML kodą,
* perkelia jį į `BeautifulSoup` objektą,
* iteruojame per eilutes ir išsitraukiame reikiamą informaciją,
* duomenis sukeliame į `pandas` DataFrame objektą ir jį grąžiname.

In [2]:
#@title CVonline funkcija
async def cvonline(keyword="python"):
    """
    This function returns all available job listings based on search keyword.
    Inputs:
      keyword (str)
    Output:
      returns pandas DataFrame
    """
    async with async_playwright() as p:

        # create webdriver/webkit
        browser = await p.webkit.launch()

        # create user agent for the webdriver
        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0'

        # create new page, i.e. new table in your browser
        page = await browser.new_page(user_agent=user_agent)
        
        # generate URL with a keyword
        url = f"https://cvonline.lt/lt/search?limit=500&offset=0&keywords%5B0%5D={keyword}&fuzzy=true&suitableForRefugees=false&isHourlySalary=false&isRemoteWork=false&isQuickApply=false"
        
        # visit page
        await page.goto(url)

        # click on pop-up window
        await page.click("//button[@class='jsx-4189752321 close-modal-button']")

        # click cookie button
        await page.click("//button[@class='cookie-consent-button']")

        # imlicit wait
        time.sleep(2)

        # get page html contents
        page_source = await page.content()

        # convert to bs4 object
        soup = BeautifulSoup(page_source, "lxml")

        # find all rows (ul - unordered list, li - list item)
        rows = soup.find("ul", {"data-gtm-id": "search-results"}).find_all("li")

        # create tmp. list to store data
        lst = list()

        # iterate over all rows
        for row in rows:
          # find all <a> tags
          for a in row.find_all('a', href=True):
            # condition to find employr info
            if "employer" in a['href']:
              # get element's text
              employer = a.text
          
          # get all row contents
          _contents = row.find_all("span")

          # get specific info about job title and job location
          job_title = _contents[0].text
          job_location = _contents[2].text[3:]

          # get start day (the date job was created)
          for c in _contents[4:]:
            # if it started
            if "Paskelbta" in c.text:
              # get text value
              offer_started = c.text.split("Baigiasi")[0]
            # if job offered is closed
            if "Baigiasi" in c.text:
              # get text value
              offer_ends = c.text

          # get salary value
          salary = row.find('span', {'class': 'jsx-1401030249 vacancy-item__salary-label'})

          # convert salary to text
          if salary:
            salary = salary.text
          else:
            salary = ''
    
          # "jsx-1401030249 vacancy-item__salary-label"
            

          # add data to temp. list
          lst.append([employer, job_title, job_location, offer_started, offer_ends, salary])

        # save image to your enviroment (for debuging)
        # one can close this line
        await page.screenshot(path="cvonline_status.png")
        
        # close webkit
        await browser.close()

        # return pandas DataFrame
        return pd.DataFrame(lst, columns = ['Employer', 'JobTitle', "Location", "Offered", "AddEnds", "Salary"])

In [3]:
#@title Atsisiųsti duomenis
# paieškos žodis
keyword = 'python' # @param {type:"string"}

# iškviečiame funkciją ir išsaugome duomenis ir atvaizduojame pirmus 5 skelbimus
df_cvonline = asyncio.run(cvonline(keyword))

# parašyti kiek rado skelbimų
print(f"Rado {len(df_cvonline)} skelbimų.")

df_cvonline.head()

Rado 66 skelbimų.


,Employer,JobTitle,Location,Offered,AddEnds,Salary
0,"IT Baltics, UAB",System engineer,"Vilnius, Vilniaus rajonas, Lietuva",Paskelbta prieš 7 dienas,Baigiasi: 2023.01.28,€ 3000 – 6000
1,Cognizant Lietuva,Software Engineer,"Vilnius, Vilniaus rajonas, Lietuva",Paskelbta prieš 16 dienų,Baigiasi: 2023.01.20,€ 3290 – 5720
2,Cognizant Lietuva,ETL Developer,"Vilnius, Vilniaus rajonas, Lietuva",Paskelbta prieš 21 dienas,Baigiasi: 2023.01.16,€ 2500 – 4120
3,Cognizant Lietuva,Data Engineer,"Vilnius, Vilniaus rajonas, Lietuva",Paskelbta prieš 16 dienų,Baigiasi: 2023.01.20,€ 3290 – 5720
4,"UAB, Dropstone",Pardavimų konsultantas (-ė),"Vilnius, Vilniaus rajonas, Lietuva",Paskelbta prieš 1 dieną,Baigiasi: 2023.02.03,€ 1160.16 – 1544.02


# Duomenų apdorojimas

Duomenis būtina sutvarkyti prieš pradedant analizuoti. Atlyginimo stulpelis `salary` turi keletą tipų reikšmių:

* vieni atlyginimai parašyti per ruožą, pvz. € 3300 – 4000. Tokiu atveju, reikia ištraukti minimalią ir maksimalią atlyginimo vertes, panaikinti euro simbolį.
* kiti skelbimai neskelbia atlygimų, tiesiog rašo "TOP Darbdavys". Tokius įrašus reikia paversti NaN vertėmis.
* yra atlyginimų, kur rašo valandinį, pvz. € 6/h, tokiu šį atlyginimą paversti į mnesinį.

Galiausiai atlyginimai yra sunormuojami į vidurkį tarp minimalaus ir maksimalaus siūlomo varianto.

In [4]:
#@title Sutvarkyti skaitinius duomenis
def clean_num_cols(df):
  """
  Formats salary columns
  Input:
    df - pandas DataFrame
  Output:
    pandas DataFrame
  """
  # clean empty salaries (the ones with Top darbdavys)
  df.Salary = df.Salary.apply(lambda x: "" if "TOP Darbdavys" in x else x)

  # if there is salary range, e.g. x - y, then extract min and max values
  df['SalaryMin'] = df.Salary.apply(lambda x: x.split(" – ")[0][2:] if " – " in x else x)
  df['SalaryMax'] = df.Salary.apply(lambda x: x.split(" – ")[1] if " – " in x else x)

  # remove euro sign
  df['SalaryMin'] = df['SalaryMin'].str.replace("€", "")
  df['SalaryMax'] = df['SalaryMax'].str.replace("€", "")

  # assume that each month has 22 working days with 8 hours a day
  # approximate hourly wages to monthly
  # condition to select rows
  cond = df.Salary.apply(lambda x: "/h" in x)

  # convert hourly data to month salaries
  df.loc[cond, 'SalaryMin'] = df.loc[cond, 'Salary'].apply(lambda x: float(x.split("/h")[0].replace("€", "")) * 22 * 8)
  df.loc[cond, 'SalaryMax'] = df.loc[cond, 'Salary'].apply(lambda x: float(x.split("/h")[0].replace("€", "")) * 22 * 8)

  # convert missing salaries to NaNs
  df.loc[df.SalaryMin == '', 'SalaryMin'] = np.nan
  df.loc[df.SalaryMax == '', 'SalaryMax'] = np.nan

  # convert to floats
  df['SalaryMin'] = df['SalaryMin'].astype(float)
  df['SalaryMax'] = df_cvonline['SalaryMax'].astype(float)

  # calculate average salary
  df['SalaryMean'] = (df['SalaryMin'] + df['SalaryMax']) / 2
  
  # return cleaned DataFrame
  return df


In [5]:
# clean numerical values
df_c = clean_num_cols(df_cvonline)

# drop adds without salary
df_c = df_c.dropna()
df_c.head()

,Employer,JobTitle,Location,Offered,AddEnds,Salary,SalaryMin,SalaryMax,SalaryMean
0,"IT Baltics, UAB",System engineer,"Vilnius, Vilniaus rajonas, Lietuva",Paskelbta prieš 7 dienas,Baigiasi: 2023.01.28,€ 3000 – 6000,3000.00,6000.00,4500.00
1,Cognizant Lietuva,Software Engineer,"Vilnius, Vilniaus rajonas, Lietuva",Paskelbta prieš 16 dienų,Baigiasi: 2023.01.20,€ 3290 – 5720,3290.00,5720.00,4505.00
2,Cognizant Lietuva,ETL Developer,"Vilnius, Vilniaus rajonas, Lietuva",Paskelbta prieš 21 dienas,Baigiasi: 2023.01.16,€ 2500 – 4120,2500.00,4120.00,3310.00
3,Cognizant Lietuva,Data Engineer,"Vilnius, Vilniaus rajonas, Lietuva",Paskelbta prieš 16 dienų,Baigiasi: 2023.01.20,€ 3290 – 5720,3290.00,5720.00,4505.00
4,"UAB, Dropstone",Pardavimų konsultantas (-ė),"Vilnius, Vilniaus rajonas, Lietuva",Paskelbta prieš 1 dieną,Baigiasi: 2023.02.03,€ 1160.16 – 1544.02,1160.16,1544.02,1352.09


# Įžvalgos

Keletas klausimų į kurios galima atsakyti tiek vizualiai tiek skaičiais.

In [6]:
#@title Vidutinis atlyginimas

# atspausindit atsakymą
print(f'Pagal pieškos žodį "{keyword}"" buvo {len(df_c)} skelbimai vidutiškai siūlo {df_c.SalaryMean.mean():.0f} € atlyginimą')

Pagal pieškos žodį "python"" buvo 64 skelbimai vidutiškai siūlo 4081 € atlyginimą


In [7]:
#@title Top skelbimai su didžiausiais atlyginimais

# top N dižiausius atlyginimus turintys skelbimai
N = 10 # @param {type:"integer"}
cols = ['Employer', 'JobTitle', 'AddEnds', 'Salary', 'SalaryMean']

# sort and return N largest
df_c.sort_values(by="SalaryMean").tail(N)[cols]

,Employer,JobTitle,AddEnds,Salary,SalaryMean
42,Danske Bank Lithuania,Chief IT Software Engineer (Backend),Baigiasi: 2023.01.12,€ 4480 – 6720,5600.0
27,"Melsoft LT, UAB",C++ Client Developer,Baigiasi: 2023.01.12,€ 4200 – 7000,5600.0
56,Danske Bank Lithuania,Chief Data Engineer,Baigiasi: 2023.01.19,€ 4560 – 6840,5700.0
21,Alliance for Recruitment,DevOps/Site Reliability engineer,Baigiasi: 2023.01.30,€ 4500 – 7500,6000.0
45,Danske Bank Lithuania,Chief Service Reliability Engineer (SRE) in Me...,Baigiasi: 2023.02.10,€ 4960 – 7440,6200.0
26,Alliance for Recruitment,Technical Architect,Baigiasi: 2023.01.20,€ 5785 – 7000,6392.5
8,Alliance for Recruitment,Senior Data Architect,Baigiasi: 2023.01.26,€ 5785 – 7416,6600.5
30,Melbikomas UAB,SysOps/DevOps Team Lead,Baigiasi: 2023.01.20,€ 5800 – 7500,6650.0
32,"Revel Systems, UAB",Software Development Manager (Online Ordering ...,Baigiasi: 2023.01.19,€ 6100 – 7500,6800.0
25,"Melsoft LT, UAB",Client Tech Lead,Baigiasi: 2023.03.04,€ 6000 – 8000,7000.0


# Išsaugoti duomenis

Atkomentuoti eilutes su `CTR + /` ir išsaugoti norimu formatu. Failo pavadinimas sugenruojams:

* `cv_online_Y_X.csv` arba `cv_online_Y_X.xlsx`, kur `X` yra raktažodis, o `Y` yra data, kad buvo paleistas kodas. Jei `X` buvo iš dviejų žodžių, pvz. `duomenų analitikas`, tarpai ` ` yra paverčiami `_`.

Norint atsisiųsti duomenis lokaliai paspauskite ant dešinėje pusė esančios "Files" ikonos ir atsisiųskite norimą failą. Jei to nepadarysite, failai bus ištrinti uždarius google colab.

![image info](https://i.stack.imgur.com/mYWnb.png)




In [8]:
# generate data for today
_date = pd.Timestamp.now().strftime("%Y_%m_%d")

# excel, remove comments if you want to save data
# df_c.to_excel(f"cv_online_{keyword}_{_date}.xlsx", index = False)
# .csv, remove comments if you want to save data
# df_c.to_csv(f"cv_online_{keyword}_{_date}.csv", index = False)